# CSC 480-F25 Lab 2: Task Decomposition, Multi‑Agent Design Patterns, and Communication Protocols

# Authors:

***[Team Member 1], [Team Member 2], [Team Member 3]***

California Polytechnic State University, San Luis Obispo;

Computer Science & Software Engineering Department

# Overview

This lab focuses on:
- Breaking down complex problems into smaller tasks (task decomposition)
- Defining specialized agents with clear roles and responsibilities  
- Choosing appropriate multi-agent design patterns
- Specifying communication protocols between agents
- Implementing a three-agent system using AutoGen

NOTE: If you already know what you're going to do for your final project, you may use this lab to make progress on it. Otherwise, treat this lab as a distinct exercise.

## Learning Objectives

By the end of this lab, you will be able to:

- Decompose a complex problem into a comprehensive set of constituent tasks
- Identify and define the roles, responsibilities, and abilities of individual agents
- Select and justify an appropriate multi-agent design pattern (Manager‑Worker, Sequential Pipeline, Collaborative Team)
- Explain and apply foundational agent communication protocols at a high level (MCP, A2A)
- Start thinking about the architecture of your project (agents, tasks, design pattern, and interaction protocols)

# Part 1: Task Decomposition and Agent Design

## 1. Problem Statement & Task Decomposition

**Main Problem:** [Describe the main problem your multi-agent system will solve]

**Task Breakdown:** List the steps and sub-tasks needed to achieve the goal:
1. [Task 1]
2. [Task 2] 
3. [Task 3]
4. [Additional tasks as needed]

## 2. Agent Definition

Define your primary agents with clear roles and responsibilities:

### Agent 1: [Agent Name]
- **Role:** [Primary role/purpose]
- **Responsibilities:** [What this agent is responsible for]
- **Inputs:** [What data/information this agent receives]
- **Outputs:** [What this agent produces]
- **Success Criteria:** [How to measure if this agent succeeded]

### Agent 2: [Agent Name] 
- **Role:** [Primary role/purpose]
- **Responsibilities:** [What this agent is responsible for]
- **Inputs:** [What data/information this agent receives]
- **Outputs:** [What this agent produces]
- **Success Criteria:** [How to measure if this agent succeeded]

### Agent 3: [Agent Name]
- **Role:** [Primary role/purpose] 
- **Responsibilities:** [What this agent is responsible for]
- **Inputs:** [What data/information this agent receives]
- **Outputs:** [What this agent produces]
- **Success Criteria:** [How to measure if this agent succeeded]

## 3. Design Pattern Selection

**Chosen Pattern:** [Manager-Worker / Sequential Pipeline / Collaborative Team]

**Justification:** [Explain why this pattern fits your workflow and constraints. Consider factors like task dependencies, parallelization opportunities, coordination complexity, etc.]

## 4. Communication Design

### Model Context Protocol (MCP)
**Shared Context & Tools:** [Describe the shared context and tools all agents need - schemas, access rules, etc.]

### Agent-to-Agent (A2A) Interactions

#### Interaction 1: [Sender] → [Receiver]
- **Purpose:** [What this interaction accomplishes]
- **Key Fields:** [Important data/parameters exchanged]
- **Message Format:** [Brief description of message structure]

#### Interaction 2: [Sender] → [Receiver] 
- **Purpose:** [What this interaction accomplishes]
- **Key Fields:** [Important data/parameters exchanged]
- **Message Format:** [Brief description of message structure]

## 5. Interaction Diagram

[Create a flowchart or sequence diagram showing agents, your chosen pattern, and MCP/A2A interactions. You can use text-based diagrams, draw by hand and insert an image, or use diagramming tools.]

```
Example text-based flow:
User → Agent1 → Agent2 → Agent3 → User
  ↓       ↓       ↓       ↓
 MCP    A2A     A2A    Final
Tool   Msg1    Msg2   Output
```

# Part 2: Three-Agent AutoGen Implementation

## Environment Setup

Install required packages and set up Azure OpenAI configuration.

In [ ]:
%pip install "autogen-core" "autogen-agentchat" "autogen-ext[openai,azure]"

In [ ]:
import os
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import TextMentionTermination
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

In [ ]:
# This is the same as in L1
azure_deployment = "your-deployment-name"
api_version = "2024-12-01-preview"  # If you're using gpt-5-mini as demonstrated

# e.g. https://ccoha-mfoynknp-eastus2.cognitiveservices.azure.com/
azure_endpoint = "your-azure-endpoint"

## An Example Three-Agent System Architecture

Based on the suggested roles from the overview:
- **Planner**: Decomposes the user goal into a concise, ordered plan
- **Implementer**: Executes the steps from the plan
- **Critic/Integrator**: Reviews output and suggests improvements or approves completion

In [ ]:
async def setup_three_agent_system():
    """Set up the three-agent system with Planner, Implementer, and Critic roles"""
    
    # Create the Azure OpenAI client
    client = AzureOpenAIChatCompletionClient(
        azure_deployment=azure_deployment,
        model="gpt-5-mini",
        api_version=api_version,
        azure_endpoint=azure_endpoint,
        api_key=os.getenv("AZURE_SUBSCRIPTION_KEY"),
    )
    
    # Define the three agents with specific system prompts
    planner = AssistantAgent(
        name="Planner",
        model_client=client,
        system_message="""You are a task decomposition specialist. Your role is to:
        1. Break down complex user goals into clear, numbered steps
        2. Define specific inputs/outputs for each step  
        3. Provide testable acceptance criteria for each step
        4. Present plans in a structured, actionable format
        
        Always end your response with 'PLAN_COMPLETE' when finished."""
    )
    
    implementer = AssistantAgent(
        name="Implementer", 
        model_client=client,
        system_message="""You are an implementation specialist. Your role is to:
        1. Follow the Planner's steps methodically
        2. Address each step with concrete outputs/artifacts
        3. Provide detailed solutions that meet the specified criteria
        4. Ask for clarification if any step is unclear
        
        Always end your response with 'IMPLEMENTATION_COMPLETE' when finished."""
    )
    
    critic = AssistantAgent(
        name="Critic",
        model_client=client, 
        system_message="""You are a quality assurance specialist. Your role is to:
        1. Review the Implementer's work against the Planner's criteria
        2. Identify gaps, errors, or missing elements
        3. Suggest specific improvements if needed
        4. Only approve with 'APPROVED' if all criteria are fully met
        
        Be thorough but constructive in your feedback."""
    )
    
    return planner, implementer, critic

In [ ]:
async def run_three_agent_workflow(task_description):
    """Run the three-agent workflow for a given task"""

    # Set up the agents
    planner, implementer, critic = await setup_three_agent_system()

    # Create termination condition - look for "APPROVED" in messages
    # See https://microsoft.github.io/autogen/stable//reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TerminationCondition
    termination = TextMentionTermination("APPROVED")

    # Create a group chat with the three agents
    # The RoundRobinGroupChat will manage turn-taking between agents
    # See https://microsoft.github.io/autogen/stable//reference/python/autogen_agentchat.teams.html#autogen_agentchat.teams.RoundRobinGroupChat
    team = RoundRobinGroupChat([planner, implementer, critic], termination_condition=termination)

    # Run the workflow
    print("Starting three-agent workflow...")
    print("=" * 60)

    # Start the conversation
    result = await Console(
        team.run_stream(task=task_description)
    )

    return result

## Example Task 1: Analytical Task

Test the three-agent system with a small example analytical task.

In [ ]:
# Test Task 1: Compare sorting algorithms
analytical_task = """
Compare the pros and cons of Quick Sort vs Merge Sort algorithms. 
Provide a structured analysis that includes:
- Time and space complexity for both
- Best/worst case scenarios  
- Practical use cases for each
- A final recommendation with justification
"""

# Uncomment to run when ready:
# await run_three_agent_workflow(analytical_task)

## Example Task 2: Creative Task

Test the three-agent system with a creative writing task.

In [ ]:
# Test Task 2: Creative writing with constraints  
creative_task = """
Write a 150-word summary of the benefits of renewable energy that:
- Uses exactly 3 specific statistics or data points
- Includes at least 2 types of renewable energy sources
- Has a compelling call-to-action in the final sentence
- Maintains an optimistic but factual tone throughout
"""

# Uncomment to run when ready:
# await run_three_agent_workflow(creative_task)

## Your Task

Write your own task for the planner-implementer-critic system

In [ ]:
your_task = """
Your task description here.
"""
# Uncomment to run when ready:
# await run_three_agent_workflow(your_task)

NOTE: You'll notice that the output might contain duplicated text, that is because `Console` prints when a text message is sent and received.

## Reflection & Analysis

### What worked well?
[Reflect on where the three-agent pattern performed effectively]

### What struggled?
[Note areas where the pattern faced challenges or limitations]

### Potential improvements:
[Consider what changes could make the system more effective]

### Tool integration:
[Optional: Discuss how adding tools (like a calculator function) might improve the system]

# Optional Extensions

## Adding Tools (Optional Challenge)

Consider adding a simple tool function that agents can use, such as a calculator or text analyzer.

In [ ]:
# Example tool function (uncomment and modify as needed)
# def calculator(expression: str) -> float:
#     """Simple calculator tool for basic mathematical operations"""
#     try:
#         # WARNING: eval() should not be used in production - use a proper math parser
#         result = eval(expression)
#         return result
#     except Exception as e:
#         return f"Error: {str(e)}"

# To integrate tools with AutoGen agents, you would typically:
# 1. Define tool schemas following MCP-like patterns
# 2. Register tools with specific agents  
# 3. Update system prompts to mention available tools
# 4. Handle tool calls in the conversation flow

# Summary and Next Steps

## Key Takeaways

- **Task Decomposition**: [Your insights about breaking down complex problems]
- **Agent Design**: [What you learned about defining agent roles and responsibilities]  
- **Communication Protocols**: [Understanding of MCP and A2A interactions]

## References

- [AutoGen Documentation](https://microsoft.github.io/autogen/stable/index.html)
- [Model Context Protocol](https://modelcontextprotocol.io/docs/getting-started/intro)
- [Agent-to-Agent Protocol](https://a2a-protocol.org/latest/)
- L2_overview.md (Lab requirements and detailed explanations)